In [1]:
import copy 

import pandas as pd
import copy

import re

import os

import operator

import pymorphy2
from pymystem3 import Mystem

import nltk

import itertools 

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from collections import OrderedDict

import progressbar
import time

from collections import OrderedDict
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«"
full_punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~–,»«'

In [2]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
russian_stopwords.remove("бы")
russian_stopwords.remove("не")
russian_stopwords.remove("ни")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilyakhoang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lilyakhoang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
tokenizer = nltk.data.load('russian.pickle')
text = "Ай да А.С. Пушкин! Ай да сукин сын!"
print("Before:", nltk.sent_tokenize(text))
print("After:", tokenizer.tokenize(text))

Before: ['Ай да А.С.', 'Пушкин!', 'Ай да сукин сын!']
After: ['Ай да А.С. Пушкин!', 'Ай да сукин сын!']


In [5]:
end_line_punctuation = "!.?»"
def delete_empty_newlines(file):
    new_lines = []
    collected_line = ''
    empty_endline = False
    with open (file, "r", encoding = "utf-8") as f:
        for line in f.readlines():
            cl_line = line.strip()
            #print(cl_line, "||",cl_line[-1])
            if(cl_line[-1] not in end_line_punctuation):
                #print("go to the next")
                collected_line += ' ' + cl_line
                empty_endline = True
            else:
                if(collected_line):
                    collected_line += ' ' + cl_line
                empty_endline = False
            
            if not (empty_endline):
                if(collected_line):
                    #print(collected_line)
                    new_lines.append(collected_line)
                    collected_line = ''
                    
                else:
                    #print(cl_line)
                    new_lines.append(cl_line)
                
    return new_lines
list_sentences_lines = delete_empty_newlines("text_8.txt")

In [6]:
list_sentences_lines

['В классе был урок русского языка. Преподаватель читал новый текст.',
 'Студенты внимательно слушали.',
 '– Извините, можно войти? Я опоздал, – сказал Джон.',
 ' – Что случилось, Джон? Вчера вы опоздали, сегодня вы опоздали. Нельзя опаздывать на урок.',
 ' Вечером Джон пришел домой. Дома он долго думал: «Вчера я опоздал, сегодня я тоже опоздал. Это очень плохо. Не буду больше опаздывать на урок. Завтра я встану рано, в 8 часов, быстро сделаю зарядку, приготовлю завтрак, позавтракаю, повторю домашнее задание. Завтра я не опоздаю!»',
 ' Утром Джон встал в 8 часов. Он сделал зарядку, приготовил завтрак, позавтракал, повторил домашнее задание и пошел в институт.',
 'В 9 часов 30 минут Джон пришел в институт.',
 '– Что случилось, молодой человек? Куда вы идете? – спросил дежурный.',
 '– Я иду на урок. Сегодня я не опоздал, – ответил Джон.',
 '– Идите домой, молодой человек! Сегодня нет урока. Сегодня воскресенье.']

In [7]:
with open("alined_text_8.txt" ,"w+", encoding = 'utf-8') as f:
    for line in list_sentences_lines:
        f.write(line + '\n')

In [31]:
punct = ".?!"

In [53]:
conll_punct = full_punctuation.replace("!","")
conll_punct = conll_punct.replace("?","")
conll_punct = conll_punct.replace(".","")
conll_punct = conll_punct.replace(",","")
#conll_punct = conll_punct.replace("»","")
conll_punct

'"#$%&\'()*+-/:;<=>@[\\]^_`{|}~–»«'

In [55]:
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    clean_sentence = []
    #print(sentences)
    for sentence in sentences:
        words = sentence.split()
        clean_text= ''
        for word in words:
            clean_word = ''
            for char in word:
                if char != " " and char not in conll_punct:#full_punctuation
                    clean_word += char#.lower()
            clean_text += clean_word + ' '
        clean_text = re.sub(' +', ' ', clean_text)
        clean_text = clean_text.strip()
        clean_sentence.append(clean_text)
    return clean_sentence   

def clean_file(file_lines_list):
    preprocessed_text = []
    for line in file_lines_list:
        t = preprocess_text(line)
        #print(t)
        preprocessed_text.extend(t)

    return preprocessed_text
sentences_list = clean_file(list_sentences_lines)
sentences_list

['В классе был урок русского языка.',
 'Преподаватель читал новый текст.',
 'Студенты внимательно слушали.',
 'Извините, можно войти?',
 'Я опоздал, сказал Джон.',
 'Что случилось, Джон?',
 'Вчера вы опоздали, сегодня вы опоздали.',
 'Нельзя опаздывать на урок.',
 'Вечером Джон пришел домой.',
 'Дома он долго думал Вчера я опоздал, сегодня я тоже опоздал.',
 'Это очень плохо.',
 'Не буду больше опаздывать на урок.',
 'Завтра я встану рано, в 8 часов, быстро сделаю зарядку, приготовлю завтрак, позавтракаю, повторю домашнее задание.',
 'Завтра я не опоздаю!',
 'Утром Джон встал в 8 часов.',
 'Он сделал зарядку, приготовил завтрак, позавтракал, повторил домашнее задание и пошел в институт.',
 'В 9 часов 30 минут Джон пришел в институт.',
 'Что случилось, молодой человек?',
 'Куда вы идете?',
 'спросил дежурный.',
 'Я иду на урок.',
 'Сегодня я не опоздал, ответил Джон.',
 'Идите домой, молодой человек!',
 'Сегодня нет урока.',
 'Сегодня воскресенье.']

In [56]:
with open("text_8_sent_tokens.txt" ,"w+", encoding = 'utf-8') as f:
    for line in sentences_list:
        f.write(line + '\n')

In [7]:
def calculate_lix_from_list_of_sentences(processed_text_sentences):
        sentences_count = len(processed_text_sentences)
        words_count = sum([len(line.split(' ')) for line in processed_text_sentences])
        long_words_count = 0 #more than 6
        for line in processed_text_sentences:
            for word in line.split():
                if len(word) > 6:
                    long_words_count += 1
        lix = words_count/ sentences_count + (long_words_count * 100) / words_count
        
        return round(lix,2)
text_lix = calculate_lix_from_list_of_sentences(sentences_list)

In [24]:
m = Mystem()
al = m.analyze('в классе был урок русского языка')

In [26]:
al[2]

{'analysis': [{'gr': 'S,муж,неод=пр,ед', 'lex': 'класс', 'wt': 1}],
 'text': 'классе'}

In [8]:
sentences_list

['в классе был урок русского языка',
 'преподаватель читал новый текст',
 'студенты внимательно слушали',
 'извините можно войти',
 'я опоздал сказал джон',
 'что случилось джон',
 'вчера вы опоздали сегодня вы опоздали',
 'нельзя опаздывать на урок',
 'вечером джон пришел домой',
 'дома он долго думал вчера я опоздал сегодня я тоже опоздал',
 'это очень плохо',
 'не буду больше опаздывать на урок',
 'завтра я встану рано в 8 часов быстро сделаю зарядку приготовлю завтрак позавтракаю повторю домашнее задание',
 'завтра я не опоздаю',
 'утром джон встал в 8 часов',
 'он сделал зарядку приготовил завтрак позавтракал повторил домашнее задание и пошел в институт',
 'в 9 часов 30 минут джон пришел в институт',
 'что случилось молодой человек',
 'куда вы идете',
 'спросил дежурный',
 'я иду на урок',
 'сегодня я не опоздал ответил джон',
 'идите домой молодой человек',
 'сегодня нет урока',
 'сегодня воскресенье']

In [15]:
with open("text_8_sent_tokens.txt" ,"w+", encoding = 'utf-8') as f:
    for line in sentences_list:
        line_w = line[0].upper() + line[1:]
        print(line_w)
        f.write(line_w + '.\n')

В классе был урок русского языка
Преподаватель читал новый текст
Студенты внимательно слушали
Извините можно войти
Я опоздал сказал джон
Что случилось джон
Вчера вы опоздали сегодня вы опоздали
Нельзя опаздывать на урок
Вечером джон пришел домой
Дома он долго думал вчера я опоздал сегодня я тоже опоздал
Это очень плохо
Не буду больше опаздывать на урок
Завтра я встану рано в 8 часов быстро сделаю зарядку приготовлю завтрак позавтракаю повторю домашнее задание
Завтра я не опоздаю
Утром джон встал в 8 часов
Он сделал зарядку приготовил завтрак позавтракал повторил домашнее задание и пошел в институт
В 9 часов 30 минут джон пришел в институт
Что случилось молодой человек
Куда вы идете
Спросил дежурный
Я иду на урок
Сегодня я не опоздал ответил джон
Идите домой молодой человек
Сегодня нет урока
Сегодня воскресенье


In [113]:
def get_sent_gramm_features_map(clean_text):
    text_grammar_map = []
    m = Mystem()
    
    bar = progressbar.ProgressBar(maxval=len(clean_text),
                                      widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    line_number = 0
    bar.start()
        
    for sentence in clean_text:
        parsed_sentence_clean = []
        parsed_sentence =  m.analyze(sentence)
        for word in parsed_sentence:
            if word['text'].isalpha() or word['text'].isdigit():
                parsed_sentence_clean.append(word)
        text_grammar_map.append(parsed_sentence_clean)
        
        line_number += 1
        bar.update(line_number)
        time.sleep(0.1)
        
    return text_grammar_map
text_grammar_map_ex =  get_sent_gramm_features_map(sentences_list)

In [120]:
text_grammar_map_ex

[[{'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917878000001}],
   'text': 'в'},
  {'analysis': [{'gr': 'S,муж,неод=пр,ед', 'lex': 'класс', 'wt': 1}],
   'text': 'классе'},
  {'analysis': [{'gr': 'V,нп=прош,ед,изъяв,муж,несов',
     'lex': 'быть',
     'wt': 1}],
   'text': 'был'},
  {'analysis': [{'gr': 'S,муж,неод=(вин,ед|им,ед)',
     'lex': 'урок',
     'wt': 0.9992735223}],
   'text': 'урок'},
  {'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
     'lex': 'русский',
     'wt': 0.9496245492000001}],
   'text': 'русского'},
  {'analysis': [{'gr': 'S,муж,неод=род,ед',
     'lex': 'язык',
     'wt': 0.9963354032}],
   'text': 'языка'}],
 [{'analysis': [{'gr': 'S,муж,од=им,ед', 'lex': 'преподаватель', 'wt': 1}],
   'text': 'преподаватель'},
  {'analysis': [{'gr': 'V,несов,пе=прош,ед,изъяв,муж',
     'lex': 'читать',
     'wt': 1}],
   'text': 'читал'},
  {'analysis': [{'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)',
     'lex': 'новый',
     'wt': 

In [129]:
def lemmatize_text_from_grammar_map(text_gr_map):
    lemm_text = []
    for sentence_gr_map in text_gr_map:
        sentence = ''
        for word_grammar_features in sentence_gr_map:
            keys = list(word_grammar_features.keys())
            values = list(word_grammar_features.values())
            word  = word_grammar_features['text']
            #print(word)
            if(word.isdigit()):
                #print("digir")
                lemma = word
                sentence += lemma + ' '
            elif ('analysis' not in keys):
                pass
            elif(word_grammar_features['analysis'] ==[]):
                pass
            else:
                lemma = word_grammar_features['analysis'][0]['lex']
                sentence += lemma + ' '
        sentence = sentence.strip()
        lemm_text.append(sentence)
    return lemm_text
lemm_sentences = lemmatize_text_from_grammar_map(text_grammar_map_ex)

In [130]:
lemm_sentences

['в класс быть урок русский язык',
 'преподаватель читать новый текст',
 'студент внимательно слушать',
 'извинять можно входить',
 'я опаздывать сказать джон',
 'что случаться джон',
 'вчера вы опаздывать сегодня вы опаздывать',
 'нельзя опаздывать на урок',
 'вечер джон приходить домой',
 'дома он долго думать вчера я опаздывать сегодня я тоже опаздывать',
 'это очень плохой',
 'не быть больше опаздывать на урок',
 'завтра я вставать рано в 8 час быстро сделать зарядка приготавливать завтрак позавтракать повторять домашний задание',
 'завтра я не опаздывать',
 'утро джон вставать в 8 час',
 'он сделать зарядка приготавливать завтрак позавтракать повторять домашний задание и пойти в институт',
 'в 9 час 30 минута джон приходить в институт',
 'что случаться молодой человек',
 'куда вы идти',
 'спрашивать дежурный',
 'я идти на урок',
 'сегодня я не опаздывать отвечать джон',
 'идти домой молодой человек',
 'сегодня нет урок',
 'сегодня воскресение']

In [132]:
def get_weights_empty_list(clean_lemm_sentences_list, clean_orig_sentences_list):
    weights_list = []
    assert (len(clean_lemm_sentences_list) == len(clean_orig_sentences_list))
    
    for lemm_sentence, sentence in zip(clean_lemm_sentences_list,clean_orig_sentences_list):
        #print(lemm_sentence)
        #print(sentence)
        sentence_weights = []
        assert (len(lemm_sentence.split()) == len(sentence.split()))
        for lemm, word in zip (lemm_sentence.split(),sentence.split()) :
            
            if(lemm in russian_stopwords):
                weight = {"lemma" : lemm, "orig_word": word, "weight": 0, "stop_word":True}
            elif(lemm.isdigit()):
                weight = {"lemma" : lemm, "orig_word": word,"weight": 0, "digit":True}
            else:
                weight = {"lemma" : lemm, "orig_word": word,"weight": 0}
            sentence_weights.append(weight)
        weights_list.append(sentence_weights)
    return weights_list
text_weighs_1 = get_weights_empty_list(lemm_sentences, sentences_list)
text_weighs_1

[[{'lemma': 'в', 'orig_word': 'в', 'stop_word': True, 'weight': 0},
  {'lemma': 'класс', 'orig_word': 'классе', 'weight': 0},
  {'lemma': 'быть', 'orig_word': 'был', 'stop_word': True, 'weight': 0},
  {'lemma': 'урок', 'orig_word': 'урок', 'weight': 0},
  {'lemma': 'русский', 'orig_word': 'русского', 'weight': 0},
  {'lemma': 'язык', 'orig_word': 'языка', 'weight': 0}],
 [{'lemma': 'преподаватель', 'orig_word': 'преподаватель', 'weight': 0},
  {'lemma': 'читать', 'orig_word': 'читал', 'weight': 0},
  {'lemma': 'новый', 'orig_word': 'новый', 'weight': 0},
  {'lemma': 'текст', 'orig_word': 'текст', 'weight': 0}],
 [{'lemma': 'студент', 'orig_word': 'студенты', 'weight': 0},
  {'lemma': 'внимательно', 'orig_word': 'внимательно', 'weight': 0},
  {'lemma': 'слушать', 'orig_word': 'слушали', 'weight': 0}],
 [{'lemma': 'извинять', 'orig_word': 'извините', 'weight': 0},
  {'lemma': 'можно', 'orig_word': 'можно', 'stop_word': True, 'weight': 0},
  {'lemma': 'входить', 'orig_word': 'войти', 'wei

In [104]:
len(text_weighs_1)

25

In [30]:
def get_tf_idf_dict(lemm_text_list, save_to_csv = False):
    vect = TfidfVectorizer(stop_words = russian_stopwords)
    tfidf_matrix = vect.fit_transform(lemm_text_list)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    #print(df.head())
    if (save_to_csv): df.to_csv("./text_0_tfidf.xlsx", sep = '\t')
    tf_idf_dict = df.to_dict()
    return tf_idf_dict
tfidf_dict = get_tf_idf_dict(lemm_sentences, save_to_csv = True)

In [134]:
def assign_tf_idf(weights_list_input, tf_idf_dict):
    weights_list = copy.deepcopy(weights_list_input)
    for sentence_ind in range(len(weights_list)):
        for el_ind in range(len(weights_list[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["lemma"]
            
            if (lemma in tf_idf_dict):
                weights_list[sentence_ind][el_ind]["weight"] = tf_idf_dict[lemma][sentence_ind]
                #print(lemma, tf_idf_dict[lemma][sentence_ind])
            #else:
                #weights_list[sentence_ind][el_ind]["weight"] = 0.05
                #print(lemma, "not found")
                
    return weights_list
text_weighs_tfidf_2 = assign_tf_idf(text_weighs_1, tfidf_dict)

In [135]:
text_weighs_tfidf_2

[[{'lemma': 'в', 'orig_word': 'в', 'stop_word': True, 'weight': 0},
  {'lemma': 'класс', 'orig_word': 'классе', 'weight': 0.5361619670142973},
  {'lemma': 'быть', 'orig_word': 'был', 'stop_word': True, 'weight': 0},
  {'lemma': 'урок', 'orig_word': 'урок', 'weight': 0.37093265612787246},
  {'lemma': 'русский', 'orig_word': 'русского', 'weight': 0.5361619670142973},
  {'lemma': 'язык', 'orig_word': 'языка', 'weight': 0.5361619670142973}],
 [{'lemma': 'преподаватель', 'orig_word': 'преподаватель', 'weight': 0.5},
  {'lemma': 'читать', 'orig_word': 'читал', 'weight': 0.5},
  {'lemma': 'новый', 'orig_word': 'новый', 'weight': 0.5},
  {'lemma': 'текст', 'orig_word': 'текст', 'weight': 0.5}],
 [{'lemma': 'студент', 'orig_word': 'студенты', 'weight': 0.5773502691896257},
  {'lemma': 'внимательно',
   'orig_word': 'внимательно',
   'weight': 0.5773502691896257},
  {'lemma': 'слушать', 'orig_word': 'слушали', 'weight': 0.5773502691896257}],
 [{'lemma': 'извинять', 'orig_word': 'извините', 'weig

In [136]:
def get_text_vocabulary_20_percent_size(lemm_text_sentences_list):
    vocab = []
    for sentence in lemm_text_sentences_list:
        for word in sentence.split():
            vocab.append(word)
    vocab_set = set(vocab)
    return int(round(len(vocab_set) * 0.2,0))
text_vocab_20_percent_len = get_text_vocabulary_20_percent_size(lemm_sentences)

In [15]:
text_vocab_20_percent_len

14

In [137]:
def get_top_20_percent_words(lemm_text_sentences_list, top_n):
    """получаем топ 100 слов на основании tfidf"""
            #tfidf = TfidfVectorizer(ngram_range = {1,2})
    #vect = TfidfVectorizer(stop_words = russian_stopwords)
    #X = vect.fit_transform(lemm_text_sentences_list)
    vect = TfidfVectorizer(stop_words = russian_stopwords)
    X = vect.fit_transform(lemm_text_sentences_list)
    #print(len(X.sum(axis=0)[0]), X.sum(axis=0))
   # индексы топ 10 столбцов с максимальной суммой элементов (в столбцах)
    idx = np.ravel(X.sum(axis=0).argsort(axis=1))[::-1][:top_n]
    top_words = np.array(vect.get_feature_names())[idx].tolist()
    return top_words
top_20_percent_words = get_top_20_percent_words(lemm_sentences, text_vocab_20_percent_len)

In [140]:
def update_weights_with_top_words(weights_list_input, top_words):
    weights_list = copy.deepcopy(weights_list_input)
    for sentence_ind in range(len(weights_list)):
        for el_ind in range(len(weights_list[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["lemma"]
            
            if (lemma in top_words):
                weights_list[sentence_ind][el_ind]["top_20_word"] = True
                #print(lemma, tf_idf_dict[lemma][sentence_ind])
    return weights_list
top_words_weights_3 = update_weights_with_top_words(text_weighs_tfidf_2, top_20_percent_words)

In [141]:
top_words_weights_3[0]

[{'lemma': 'в', 'orig_word': 'в', 'stop_word': True, 'weight': 0},
 {'lemma': 'класс', 'orig_word': 'классе', 'weight': 0.5361619670142973},
 {'lemma': 'быть', 'orig_word': 'был', 'stop_word': True, 'weight': 0},
 {'lemma': 'урок',
  'orig_word': 'урок',
  'top_20_word': True,
  'weight': 0.37093265612787246},
 {'lemma': 'русский', 'orig_word': 'русского', 'weight': 0.5361619670142973},
 {'lemma': 'язык', 'orig_word': 'языка', 'weight': 0.5361619670142973}]

In [47]:
text_grammar_map_ex[0]

[{'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917878000001}],
  'text': 'в'},
 {'analysis': [{'gr': 'S,муж,неод=пр,ед', 'lex': 'класс', 'wt': 1}],
  'text': 'классе'},
 {'analysis': [{'gr': 'V,нп=прош,ед,изъяв,муж,несов', 'lex': 'быть', 'wt': 1}],
  'text': 'был'},
 {'analysis': [{'gr': 'S,муж,неод=(вин,ед|им,ед)',
    'lex': 'урок',
    'wt': 0.9992735223}],
  'text': 'урок'},
 {'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
    'lex': 'русский',
    'wt': 0.9496245492000001}],
  'text': 'русского'},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'язык', 'wt': 0.9963354032}],
  'text': 'языка'}]

In [162]:
def update_pos(weights_list, gr_map):
    
    sentences_weights_list_output = copy.deepcopy(weights_list)
    assert len(sentences_weights_list_output) == len(gr_map)#кол-во предложений
    for weights_sentence, gr_map_sentence in zip(sentences_weights_list_output, gr_map):
        assert len(weights_sentence) == len(gr_map_sentence )#кол-во слов
        for weight_word, gramm_map_word in zip(weights_sentence, gr_map_sentence):
            keys = list(gramm_map_word.keys())
            values = list(gramm_map_word.values())
            word  = gramm_map_word['text']
            #print(weight_word['word'])
            if(word.isdigit()):
                weight_word['POS'] = "number"
            elif ('analysis' not in keys):
                pass
            elif(gramm_map_word['analysis'] ==[]):
                pass
            else:
                grammar = gramm_map_word['analysis'][0]['gr']
                assert (weight_word['lemma'] == gramm_map_word['analysis'][0]['lex'])
                grammar_sep_by_comma = grammar.split(',')
                pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                if(pos):
                    weight_word['POS'] = pos
                else:
                    weight_word['POS'] = "undefined"
                    
    return sentences_weights_list_output
pos_words_weights_4 = update_pos(top_words_weights_3, text_grammar_map_ex)
pos_words_weights_4

[[{'POS': 'PR',
   'lemma': 'в',
   'orig_word': 'в',
   'stop_word': True,
   'weight': 0},
  {'POS': 'S',
   'lemma': 'класс',
   'orig_word': 'классе',
   'weight': 0.5361619670142973},
  {'POS': 'V',
   'lemma': 'быть',
   'orig_word': 'был',
   'stop_word': True,
   'weight': 0},
  {'POS': 'S',
   'lemma': 'урок',
   'orig_word': 'урок',
   'top_20_word': True,
   'weight': 0.37093265612787246},
  {'POS': 'A',
   'lemma': 'русский',
   'orig_word': 'русского',
   'weight': 0.5361619670142973},
  {'POS': 'S',
   'lemma': 'язык',
   'orig_word': 'языка',
   'weight': 0.5361619670142973}],
 [{'POS': 'S',
   'lemma': 'преподаватель',
   'orig_word': 'преподаватель',
   'weight': 0.5},
  {'POS': 'V', 'lemma': 'читать', 'orig_word': 'читал', 'weight': 0.5},
  {'POS': 'A', 'lemma': 'новый', 'orig_word': 'новый', 'weight': 0.5},
  {'POS': 'S', 'lemma': 'текст', 'orig_word': 'текст', 'weight': 0.5}],
 [{'POS': 'S',
   'lemma': 'студент',
   'orig_word': 'студенты',
   'weight': 0.577350269

In [52]:
df_sence = pd.read_csv("./for calculation/mult_sence.csv")
sence_dict = pd.Series(df_sence.count_rank.values,index=df_sence.word).to_dict()

def low(word):
    return word.lower()
df_freq = pd.read_csv("./for calculation/freq_lables.csv")
df_freq['Lemma'] = df_freq['Lemma'].apply(low)
freq_pos_dict = pd.Series(df_freq.PoS.values,index=df_freq.Lemma).to_dict()
freq_dict = pd.Series(df_freq.freq_lable.values,index=df_freq.Lemma).to_dict()

df_eng = pd.read_csv("./for calculation/eng_words.csv", sep = ' ')
eng_words = set(df_eng['orth'])

In [94]:
len(eng_words)

665

In [163]:
def create_lex_vector(lexema, pos):
    #если слова нет в часотном словаре, значит оно скорее всего довольно редкое и признаем его относящ к категории 2
    #если слово не заимствовано то признаем сложным 1 
    sence_value, freq_value, eng_value, abstract= 0,2,1,0
    diff_analysis = {}
    
    if(lexema in sence_dict):
        #print(lexema, "found")
        sence_value = sence_dict[lexema]
        
        
    if(lexema in freq_dict):
        freq_value = freq_dict[lexema]
    else:
        diff_analysis["freq_value_note"] = "not in freq dict"
    
    if(lexema in eng_words):
        eng_value = 0
    abs_suff={'аж', 'есть', 'ие', 'изм', 'изна', 'ина', 'ость', 'ота', 'ствo', 'ция',
 'чина','щина','ёж','еж'}   
    if(pos == "S"):
        for suff in abs_suff:
            if (lexema.endswith(suff)):
                abstract = 1

    diff_analysis['raw_diff_values'] = [sence_value, freq_value, eng_value, abstract]
    diff_analysis_named = OrderedDict()
    diff_analysis_named['sence_value'] = sence_value
    diff_analysis_named['freq_value'] = freq_value
    diff_analysis_named['eng_value'] = eng_value
    diff_analysis_named['abstract'] = abstract
    #diff_analysis_named = { "freq_value":freq_value, "eng_value":eng_value, "abstract":abstract, "sence_value":sence_value}

    
    return diff_analysis, diff_analysis_named

In [165]:
def update_with_lex_vector(weights_list_input):
    weights_list = copy.deepcopy(weights_list_input)
    for sentence_ind in range(len(weights_list)):
        for el_ind in range(len(weights_list[sentence_ind])):
            lemma = weights_list[sentence_ind][el_ind]["lemma"]
            if('digit' in weights_list[sentence_ind][el_ind]):
                diff_analysis['raw_diff_values'] = [0,0,0,0]
                diff_analysis_named = {"sence_value":0, "freq_value":0, "eng_value":0, "abstract":0}
            else:
                diff_analysis,diff_analysis_named  = create_lex_vector(lemma,weights_list[sentence_ind][el_ind]["POS"])
            weights_list[sentence_ind][el_ind]["lex_vector"] = {"raw": diff_analysis, "named": diff_analysis_named}
            
    return weights_list
lex_vector_weights_5 = update_with_lex_vector(pos_words_weights_4)

In [166]:
lex_vector_weights_5[0]

[{'POS': 'PR',
  'lemma': 'в',
  'lex_vector': {'named': OrderedDict([('sence_value', 2),
                ('freq_value', 0),
                ('eng_value', 1),
                ('abstract', 0)]),
   'raw': {'raw_diff_values': [2, 0, 1, 0]}},
  'orig_word': 'в',
  'stop_word': True,
  'weight': 0},
 {'POS': 'S',
  'lemma': 'класс',
  'lex_vector': {'named': OrderedDict([('sence_value', 1),
                ('freq_value', 0),
                ('eng_value', 1),
                ('abstract', 0)]),
   'raw': {'raw_diff_values': [1, 0, 1, 0]}},
  'orig_word': 'классе',
  'weight': 0.5361619670142973},
 {'POS': 'V',
  'lemma': 'быть',
  'lex_vector': {'named': OrderedDict([('sence_value', 2),
                ('freq_value', 0),
                ('eng_value', 1),
                ('abstract', 0)]),
   'raw': {'raw_diff_values': [2, 0, 1, 0]}},
  'orig_word': 'был',
  'stop_word': True,
  'weight': 0},
 {'POS': 'S',
  'lemma': 'урок',
  'lex_vector': {'named': OrderedDict([('sence_value', 2),
         

In [53]:
m.analyze("не идут это")

[{'analysis': [{'gr': 'PART=', 'lex': 'не', 'wt': 1}], 'text': 'не'},
 {'text': ' '},
 {'analysis': [{'gr': 'V,несов,нп=непрош,мн,изъяв,3-л',
    'lex': 'идти',
    'wt': 1}],
  'text': 'идут'},
 {'text': ' '},
 {'analysis': [{'gr': 'SPRO,ед,сред,неод=(вин|им)',
    'lex': 'это',
    'wt': 0.7809833731}],
  'text': 'это'},
 {'text': '\n'}]

In [167]:
def update_coreference_negation(weight_list):
    weight_list_output = copy.deepcopy(weight_list)
    coreference_list = ["который","которая","это","этот","эти","которое"]
    negation_list = ["не","ни","бы"]
    all_sentences_map = []
    for sentence in weight_list_output:
        sentence_map = {'words':[],"negation_items":[],"coreference_items":[]}
        for word_weight in sentence:
            #print(word_weight)
            if word_weight['lemma'] in negation_list:
                sentence_map['negation_items'].append(word_weight['lemma'])
                word_weight['negation'] = True
            elif word_weight['lemma'] in coreference_list:
                sentence_map['coreference_items'].append(word_weight['lemma'])
                word_weight['coreference'] = True
            sentence_map['words'].append(word_weight)
        all_sentences_map.append(sentence_map)
    return all_sentences_map

sts_map = update_coreference_negation(lex_vector_weights_5)
for sent in sts_map:
    print (sent)
    print()

{'words': [{'lemma': 'в', 'orig_word': 'в', 'weight': 0, 'stop_word': True, 'POS': 'PR', 'lex_vector': {'raw': {'raw_diff_values': [2, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}, {'lemma': 'класс', 'orig_word': 'классе', 'weight': 0.5361619670142973, 'POS': 'S', 'lex_vector': {'raw': {'raw_diff_values': [1, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 1), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}, {'lemma': 'быть', 'orig_word': 'был', 'weight': 0, 'stop_word': True, 'POS': 'V', 'lex_vector': {'raw': {'raw_diff_values': [2, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}, {'lemma': 'урок', 'orig_word': 'урок', 'weight': 0.37093265612787246, 'top_20_word': True, 'POS': 'S', 'lex_vector': {'raw': {'raw_diff_values': [2, 1, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 1), ('eng_value', 1), ('abstract', 0)])}}, {'lemma': 'рус

In [74]:
m.analyze("мыть")

[{'analysis': [{'gr': 'V,несов,пе=инф', 'lex': 'мыть', 'wt': 1}],
  'text': 'мыть'},
 {'text': '\n'}]

In [77]:
m.analyze("мыться")

[{'analysis': [{'gr': 'V,несов,нп=инф', 'lex': 'мыться', 'wt': 1}],
  'text': 'мыться'},
 {'text': '\n'}]

In [75]:
m.analyze("делая")

[{'analysis': [{'gr': 'V,несов,пе=непрош,деепр', 'lex': 'делать', 'wt': 1}],
  'text': 'делая'},
 {'text': '\n'}]

In [76]:
m.analyze("делавший")

[{'analysis': [{'gr': 'V,несов,пе=(прош,вин,ед,прич,полн,муж,действ,неод|прош,им,ед,прич,полн,муж,действ)',
    'lex': 'делать',
    'wt': 1}],
  'text': 'делавший'},
 {'text': '\n'}]

In [38]:
len(text_grammar_map_ex[0]), len(sts_map[0]['words'] )

(6, 6)

In [39]:
len(text_grammar_map_ex), len(sts_map)

(25, 25)

In [168]:
def update_weight(s_weight, w_weight, pos):
    if ("special_pos" in w_weight):
        w_weight['special_pos'].append(pos)
    else:
        w_weight['special_pos'] = []
        w_weight['special_pos'].append(pos)
    s_weight['sent_special_pos'].append(w_weight['lemma'] + '_' + pos)

def update_special_pos(sentences_weights_list, gr_map):
    special_pos_list = ["инф", "прич", "деепр"]
    
    sentences_weights_list_output = copy.deepcopy(sentences_weights_list)
    assert len(sentences_weights_list_output) == len(gr_map)#кол-во предложений
    for weights_sentence, gr_map_sentence in zip(sentences_weights_list_output, gr_map):
        assert len(weights_sentence['words']) == len(gr_map_sentence )#кол-во слов
        weights_sentence['sent_special_pos'] = []
        for weight_word, gramm_map_word in zip(weights_sentence['words'], gr_map_sentence):
            keys = list(gramm_map_word.keys())
            values = list(gramm_map_word.values())
            word  = gramm_map_word['text']
            #print(weight_word['word'])
            if ('analysis' not in keys):
                pass
            elif(gramm_map_word['analysis'] ==[]):
                pass
            else:
                grammar = gramm_map_word['analysis'][0]['gr']
                assert (weight_word['lemma'] == gramm_map_word['analysis'][0]['lex'])
                for spec_pos in special_pos_list:
                    if (spec_pos in grammar):
                        update_weight(weights_sentence, weight_word,spec_pos)
                grammar_sep_by_comma = grammar.split(',')
                if (len(grammar_sep_by_comma) == 1):
                    pass
                else:
                    pos = re.match('[A-Z\s]+', grammar_sep_by_comma[0])[0]
                    if(pos == "V"):
                        if(gramm_map_word["text"].endswith("сь") or gramm_map_word["text"].endswith("ся")):
                            update_weight(weights_sentence, weight_word,"возвратный")
    return sentences_weights_list_output
sts_map_spec_2 = update_special_pos(sts_map, text_grammar_map_ex)

In [169]:
for sentence in sts_map_spec_2:
    print("sentence")
    print(sentence['negation_items'],sentence['coreference_items'],sentence['sent_special_pos'])
    for word in sentence['words']:
        print(word)
        print ( )
    print("==================================================")

sentence
[] [] []
{'lemma': 'в', 'orig_word': 'в', 'weight': 0, 'stop_word': True, 'POS': 'PR', 'lex_vector': {'raw': {'raw_diff_values': [2, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}

{'lemma': 'класс', 'orig_word': 'классе', 'weight': 0.5361619670142973, 'POS': 'S', 'lex_vector': {'raw': {'raw_diff_values': [1, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 1), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}

{'lemma': 'быть', 'orig_word': 'был', 'weight': 0, 'stop_word': True, 'POS': 'V', 'lex_vector': {'raw': {'raw_diff_values': [2, 0, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 0), ('eng_value', 1), ('abstract', 0)])}}

{'lemma': 'урок', 'orig_word': 'урок', 'weight': 0.37093265612787246, 'top_20_word': True, 'POS': 'S', 'lex_vector': {'raw': {'raw_diff_values': [2, 1, 1, 0]}, 'named': OrderedDict([('sence_value', 2), ('freq_value', 1), ('eng_value', 1), ('abstract', 0)])}}

{'lemma

In [41]:
text_grammar_map_ex[0]

[{'analysis': [{'gr': 'PR=', 'lex': 'в', 'wt': 0.9999917878}], 'text': 'в'},
 {'analysis': [{'gr': 'S,муж,неод=пр,ед', 'lex': 'класс', 'wt': 1}],
  'text': 'классе'},
 {'analysis': [{'gr': 'V,нп=прош,ед,изъяв,муж,несов', 'lex': 'быть', 'wt': 1}],
  'text': 'был'},
 {'analysis': [{'gr': 'S,муж,неод=(вин,ед|им,ед)',
    'lex': 'урок',
    'wt': 0.9992735223}],
  'text': 'урок'},
 {'analysis': [{'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)',
    'lex': 'русский',
    'wt': 0.9496245492}],
  'text': 'русского'},
 {'analysis': [{'gr': 'S,муж,неод=род,ед', 'lex': 'язык', 'wt': 0.9963354032}],
  'text': 'языка'}]

In [93]:
len(sts_map[0]["words"])

6

In [85]:
sts_map_spec_2[0]

{'coreference_items': [],
 'negation_items': [],
 'sent_special_pos': [],
 'words': [{'POS': 'undefined',
   'lex_vector': {'named': OrderedDict([('sence_value', 2),
                 ('freq_value', 0),
                 ('eng_value', 1),
                 ('abstract', 0)]),
    'raw': {'raw_diff_values': [2, 0, 1, 0]}},
   'stop_word': True,
   'weight': 0,
   'word': 'в'},
  {'POS': 'S',
   'lex_vector': {'named': OrderedDict([('sence_value', 1),
                 ('freq_value', 0),
                 ('eng_value', 1),
                 ('abstract', 0)]),
    'raw': {'raw_diff_values': [1, 0, 1, 0]}},
   'weight': 0.5361619670142973,
   'word': 'класс'},
  {'POS': 'V',
   'lex_vector': {'named': OrderedDict([('sence_value', 2),
                 ('freq_value', 0),
                 ('eng_value', 1),
                 ('abstract', 0)]),
    'raw': {'raw_diff_values': [2, 0, 1, 0]}},
   'stop_word': True,
   'weight': 0,
   'word': 'быть'},
  {'POS': 'S',
   'lex_vector': {'named': OrderedDict([

# ФИЧИ ТЕКСТОВЫЕ - финальное состояние единицы текста

In [ ]:
на уровен предложения

sent_special_pos СЛОВО_инф", "СЛОВО_прич", "СЛОВО_деепр СЛОВО_возвратный")
отдельно к глаголам
все вместе к глаголам

negation_items: само слово
    все вместе ко всем НЕ СТОП словам
coreference_items: само слово
    все вместе ко всем НЕ СТОП словам
    

In [170]:
def caclulate_overall_text_features(sentences_map, processed_text_sentences, lemm_text_sentences, raw_aligned_text):
    text_map = {}
    non_stop_word_count = 0
    verbs_count = 0
    inf, prich, deeprich, vozvr = 0,0,0,0
    all_spec_pos = 0
    negation_count, coreference_count = 0,0
    
    
    for sentence in sentences_map:
        for spec_pos in sentence['sent_special_pos']:
            if("_инф" in spec_pos):
                
                inf += 1
                all_spec_pos += 1
            elif("_прич" in spec_pos):
                
                prich += 1
                all_spec_pos += 1
            elif("_деепр" in spec_pos):
                
                deeprich += 1
                all_spec_pos += 1
            elif("_возвратный" in spec_pos):
                
                vozvr += 1
        negation_count += len(sentence['negation_items'])      
        coreference_count += len(sentence['coreference_items']) 
        
        for word in sentence['words']:
            #print(word)
            if (word["POS"] == "V"):
                verbs_count += 1
            if("stop_word" not in word):
                non_stop_word_count += 1
                
        
    text_map = OrderedDict()
    text_map['lix'] = calculate_lix_from_list_of_sentences(processed_text_sentences)
    text_map['ttr'] = calculate_type_token_ratio(lemm_text_sentences)
    text_map['inf_verb'] = inf/verbs_count
    text_map['prich_verb'] = prich/verbs_count
    text_map['deeprich_verb'] = deeprich/verbs_count
    text_map['vozvr_verb'] = vozvr/verbs_count
    text_map['spec_pos_verb'] = all_spec_pos/ verbs_count
    
    text_map['negation_per_n_stop_words'] = negation_count/non_stop_word_count
    text_map ['coreference_per_n_stop_words'] = coreference_count/non_stop_word_count
    
    text_map['sentences&words_analysis'] = sentences_map
    
    text_map['raw_aligned_text'] = raw_aligned_text
    text_map['cleaned_sentences'] = processed_text_sentences
    text_map['lemm_sentences'] = lemm_text_sentences
    
    return  text_map
caclulate_overall_text_features(sts_map_spec_2, sentences_list,lemm_sentences, list_sentences_lines)

OrderedDict([('lix', 40.66),
             ('ttr', 0.53),
             ('inf_verb', 0.08108108108108109),
             ('prich_verb', 0.0),
             ('deeprich_verb', 0.0),
             ('vozvr_verb', 0.05405405405405406),
             ('spec_pos_verb', 0.08108108108108109),
             ('negation_per_n_stop_words', 0.0297029702970297),
             ('coreference_per_n_stop_words', 0.009900990099009901),
             ('sentences&words_analysis',
              [{'coreference_items': [],
                'negation_items': [],
                'sent_special_pos': [],
                'words': [{'POS': 'PR',
                  'lemma': 'в',
                  'lex_vector': {'named': OrderedDict([('sence_value', 2),
                                ('freq_value', 0),
                                ('eng_value', 1),
                                ('abstract', 0)]),
                   'raw': {'raw_diff_values': [2, 0, 1, 0]}},
                  'orig_word': 'в',
                  'stop_word':

In [ ]:
def calc_common_features(text_map_input,processed_text_sentences, lemm_text_sentences ):
    lix = lemm_text_sentences()

In [ ]:
ЗДЕСЬ ЖЕ РАССЧИТАТЬ ЛЕКСИЧЕСКИЙ ВЕКТОР ЕТКСТА (ОПУСКАТЬ СТОП СЛОВА)

In [94]:
def calculate_lix_from_list_of_sentences(processed_text_sentences):
        sentences_count = len(processed_text_sentences)
        words_count = sum([len(line.split(' ')) for line in processed_text_sentences])
        long_words_count = 0 #more than 6
        for line in processed_text_sentences:
            for word in line.split():
                if len(word) > 6:
                    long_words_count += 1
        lix = words_count/ sentences_count + (long_words_count * 100) / words_count
        
        return round(lix,2)
text_lix = calculate_lix_from_list_of_sentences(sentences_list)
text_lix

40.66

In [96]:
def calculate_type_token_ratio(lemm_text_sentences):
      all_words = []
      for sentence in lemm_text_sentences:
          words = sentence.split()
          for word in words:
              all_words.append(word)

      unqie_words = set(all_words)
      types = len(unqie_words)
      tokens = len (all_words)

      return round(types/tokens,2)
text_ttr = calculate_type_token_ratio(lemm_sentences)
text_ttr

0.52

# ИДЕЯ - ТЕСТРОВАТЬ ЛЕКСИКУ В ТЕКСТЕ. ПРОЙТИСЬ ПО ВСЕМ ТЕКСТАМ И НАРЫТЬ ТАКИЕ ТРИ ТЕКСТА КОТОРЫЕ КЛЮЧАТ В СЕБЯ ВСЕ НЕОБХОДИМЫЕ СЛОВА

ЗДЕСЬ МОЖЕТ ПОНАДОБИТЬСЯ ТОП 20% слов